In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv' , index_col='id')

df

In [ ]:
X = df.drop(columns=['target'] , axis=1)
y = df['target']

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=100).fit(X)
# print(pca.explained_variance_ratio_)
# print(pca.singular_values_)
PCAX = pca.transform(X)
pd.DataFrame(PCAX)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

im = RandomOverSampler(sampling_strategy = 'minority')

resampled_X , resampled_y = im.fit_resample(PCAX , y)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(resampled_y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(resampled_X, resampled_y ,test_size=0.2 ,random_state=2)
print(X_train.shape , X_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import RidgeClassifier ,Lasso 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
model = SVC(degree=2 , C=0.7 , kernel='linear')

model.fit(X_train , y_train)
print('training score = ' , roc_auc_score(y_true =y_train ,y_score=model.predict(X_train)) * 100 , '\t' 
 , 'testing score  = ' , roc_auc_score(y_true =y_test  ,y_score=model.predict(X_test )) * 100)

In [ ]:
test_X = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv' , index_col='id')
test_PCAX = pca.transform(test_X)
test_PCAX
arr = model.predict(test_PCAX)

In [ ]:
out = pd.DataFrame(arr , index = test_X.index , columns=['target'])
out.to_csv('submission.csv')
out